In [5]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['pandas', 'psycopg2-binary', 'spacy']
pipmain(['install'] + packages)

In [7]:
%%capture
from nltk.tokenize import casual_tokenize
import nltk
#nltk.download('averaged_perceptron_tagger') #uncomment this out to download

import pandas as pd
import psycopg2
import spacy
from spacy.matcher import PhraseMatcher
import re
from collections import Counter

In [8]:
%%capture
!python -m spacy download en_core_web_lg

In [9]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
#nltk.download('stopwords') #uncomment this out to download
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

Note you need a file with the aws credentials to access the database.   
Here, I am using credentials stored in a file called `aws.py` with the following format:
```
host = 'endpoint_url_string'
port = port_number
user = 'user_string'
password = 'password_string'
database = 'db_name_string'
```

[Amazon RDS — Step-By-Step Guide](https://towardsdatascience.com/amazon-rds-step-by-step-guide-14f9f3087d28)

In [12]:
from aws import host as ahost, port as aport, user as auser, password as apassword, database as adatabase

connection = psycopg2.connect(host = ahost, 
                              port = aport, 
                              user = auser, 
                              password = apassword, 
                              dbname = adatabase)
cursor = connection.cursor()

In [13]:
def read_try(sql):
    try:
        df = pd.read_sql(sql, con=connection)
        return pd.DataFrame() if df.empty else df
    except Exception as e:
        #print("READ ERROR", e)
        return pd.DataFrame()
        
# get all the users from the users table
def read_document_tags_table_from_db():
    sql = 'SELECT * FROM document_tags'
    return read_try(sql)

In [14]:
document_tags = read_document_tags_table_from_db()

In [15]:
document_tags

,allegation_id,title,document_text,url,text_bad,incident_date,allegation_category,allegation_name,organization,nudity_penetration,sexual_harassment_remarks,sexual_humiliation_extortion_or_sex_work,tasers,trespass,racial_slurs,planting_drugs_guns,neglect_of_duty,refuse_medical_assistance,irrational_aggressive_unstable,searching_arresting_minors
0,315627,CRID 315627 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-09-20,Operation/Personnel Violations,Misuse Of Department Equipment / Supplies,CPB,None,None,None,None,None,None,None,None,None,None,None
1,315524,CRID 315524 CR,SUMMARY REPORT DIGEST- COMPLAINT REGISTER INVE...,https://assets.documentcloud.org/documents/440...,True,2006-08-24,Operation/Personnel Violations,Neglect Of Duty,CR,None,None,None,None,None,None,None,None,None,None,None
2,315452,CRID 315452 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-08-31,Operation/Personnel Violations,Neglect Of Duty,CPB,None,None,None,None,None,None,None,None,None,None,None
3,315343,CRID 315343 CR,COMMAND CHANNEL REVIEW A Sasso Emp 14982 NW 1 ...,https://assets.documentcloud.org/documents/457...,None,2006-07-21,False Arrest,Illegal Arrest / False Arrest,CR,False,False,False,False,False,False,False,False,False,False,False
4,315047,CRID 315047 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-06-11,Traffic,Improper Processing / Reporting / Procedures,CPB,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,1000436,CRID 1000436 CR,REPORT DIGEST LOG NO. TYPE DATE OF REPORT CHI...,https://assets.documentcloud.org/documents/476...,None,2006-04-21,Operation/Personnel Violations,Eeo Investigations,CR,None,None,None,None,None,None,None,None,None,None,None
1679,1000417,CRID 1000417 CR,COMMAND CHANNEL REVIEW COMPLAINT REGISTER INVE...,https://assets.documentcloud.org/documents/457...,None,2006-10-06,False Arrest,Illegal Arrest / False Arrest,CR,None,None,None,None,None,None,None,None,None,None,None
1680,1000301,CRID 1000301 CR,7/18/2018 View Incident Print Logout Help ...,https://assets.documentcloud.org/documents/476...,None,2006-10-07,Operation/Personnel Violations,Neglect Of Duty,CR,None,None,None,None,None,None,None,None,None,None,None
1681,1000227,CRID 1000227 CR (Discovery),1000227 COMMAND CHANNEL REVIEW-SUSTAINED CASE ...,https://assets.documentcloud.org/documents/522...,None,2006-05-22,Operation/Personnel Violations,Misuse Of Department Equipment / Supplies,CR,None,None,None,None,None,None,None,None,None,None,None


Document tags we are looking for:   
(List from Piazza)
- Nudity, Penetration
- sexual harassment, Sexual remarks
- Sexual humiliation, Sexual extortion, Prostitution/sex work
- Tasers, Baton, aggressive physical touch, gun
- Trespass/robbery
- Racial slurs, xenophobic remarks
- Undocumented status- asking for someone’s status, Calling ICE
- Planting drug/guns
- Neglect of duty, failure to serve
- Refusing to provide medical assistance
- “Irrational”, “Aggressive”, “Unstable”
- searching/patting down/arresting minors

In [16]:
test_index = 3
text1 = document_tags['document_text'][test_index]
print(text1)

pdf1 = document_tags['url'][test_index]

COMMAND CHANNEL REVIEW A Sasso Emp 14982 NW 1 32007 DATE INITIATED COMPLAINT REGISTER INVESTIGATION No.2 CHICAGO POLICE DEPARTMENT 315343 12 MARCH 200? INVESTIGATOR SECTION T0: UNIT: Commanding Of?cer Unit IZI-GIS PLEASE SUBMIT THIS COMMAND CHANNEL REVIEW FORM THROUGH CHANNELS WITH THE CASE FILE TO: INTERNAL AFFAIRS DIVISION OFFICE OF PROFESSIONAL STANDARDS DATE RECEIVED BY UNIT I CONCUR [a RECOMMEND OPTIONS BE GRANTED NOT RECOMMEND OPTIONS NOT BE GRANTED REVIEWED BY TITLE A  UNIT - DATE FORWARDED BY UNIT 0 ?Wrdy a 7 DATE RECEIVED BY UNIT  I CONCUR I DO NOT  14/4457: 70%? RECOMMEND OPTIONS BE GRANTED CI RECOMMEND OPTIONS NOT BE GRANTED REVIEWED BYZ UNIT 27/ Maj/6;? 4/ Dem/{Z/ 54?  DATE FORWARDED BY UNIT DATE RECEIWY UNIT [3 I CONCUR RECOMMEND OPTIONS NOT BE GRANTED I DO NOT RECOMMEND OPTIONS SE GRANTED REVIEWED BY I TITLE UNIT I I DATE FORWARDED BY UNIT DATE RECEIVED BY UNIT I CONCUR RECOMMEND OPTIONS NOT BE GRANTED I DO NOT RECOMMEND OPTIONS BE GRANTED REVIEWED BY TITLE UNIT DATE FORW

In [18]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [19]:
print(pdf1)
PDF(pdf1,size=(1000,450))

https://assets.documentcloud.org/documents/4575428/CAR-CR315343-Redacted.pdf


As we can see from the text and the pdf (retrieved from the url), the text version of the pdf is terribly unclean.

- [spaCy text classifier](https://spacy.io/usage/examples#textcat)

In [20]:
# tokenize
def get_tokens(text):
    doc = nlp(text)
    return [token.orth_ for token in doc if not token.is_punct | token.is_space] 

# split by puncuation
def split_by_punc(text):
    return [x.strip() for x in re.split('[,/]', text)]

# split by puncuation and whitespace
# also remove stopwords
def split_by_space_punc(text):
    words = [x.strip() for x in re.split('\W+', text)]
    new_words = []
    for w in words:
        if not nlp.vocab[w].is_stop:
            new_words.append(w)
    return new_words

In [21]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched
terminology_list = ["Nudity, Penetration", 
                   "sexual harassment, Sexual remarks", 
                   "Sexual humiliation, Sexual extortion, Prostitution/sex work",
                   "Taser, Baton, physical touch, gun",
                   "Trespass/robbery, Racial slurs, xenophobic remarks",
                   "Undocumented status, ICE",
                   "Planting drug, planting guns",
                   "Neglect of duty, failure to serve",
                   "Refusing to provide medical assistance",
                   "Irrational, Aggressive, Unstable",
                   "searching minors/patting down minors/arresting minors"]
# split by punctuation and spaces

In [22]:
for t in terminology_list:
    print(t, ": ", split_by_punc(t))

Nudity, Penetration :  ['Nudity', 'Penetration']
sexual harassment, Sexual remarks :  ['sexual harassment', 'Sexual remarks']
Sexual humiliation, Sexual extortion, Prostitution/sex work :  ['Sexual humiliation', 'Sexual extortion', 'Prostitution', 'sex work']
Taser, Baton, physical touch, gun :  ['Taser', 'Baton', 'physical touch', 'gun']
Trespass/robbery, Racial slurs, xenophobic remarks :  ['Trespass', 'robbery', 'Racial slurs', 'xenophobic remarks']
Undocumented status, ICE :  ['Undocumented status', 'ICE']
Planting drug, planting guns :  ['Planting drug', 'planting guns']
Neglect of duty, failure to serve :  ['Neglect of duty', 'failure to serve']
Refusing to provide medical assistance :  ['Refusing to provide medical assistance']
Irrational, Aggressive, Unstable :  ['Irrational', 'Aggressive', 'Unstable']
searching minors/patting down minors/arresting minors :  ['searching minors', 'patting down minors', 'arresting minors']


In [23]:
for t in terminology_list:
    terms = split_by_punc(t)
    patterns = [nlp.make_doc(text) for text in terms]
    matcher.add(t, None, *patterns)

In [24]:
testtext = "Taser, Baton, physical touch, gun"

In [25]:
doc = nlp(text1)
matches = matcher(doc)#print the matched results and extract out the results
for match_id, start, end in matches:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]  # The matched span
    #print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))
    print("Category: {}     Word: {}".format(string_id, span.text))

Category: Irrational, Aggressive, Unstable     Word: irrational
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: ROBBERY
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: Trespass


In [26]:
tokens_text1 = get_tokens(text1)
Counter(tokens_text1).most_common()

[('the', 97),
 ('1', 75),
 ('No', 70),
 ('of', 64),
 ('Date', 54),
 ('I', 48),
 ('to', 47),
 ('NO', 37),
 ('or', 35),
 ('3', 34),
 ('Name', 33),
 ('Arrest', 33),
 ('and', 31),
 ('a', 29),
 ('Of?cer', 28),
 ('5', 27),
 ('UNIT', 26),
 ('2', 26),
 ('DATE', 22),
 ('at', 22),
 ('that', 21),
 ('CPD', 20),
 ('OF', 19),
 ('complainant', 19),
 ('011', 18),
 ('was', 18),
 ('by', 18),
 ('7', 15),
 ('Police', 15),
 ('Page', 15),
 ('TO', 14),
 ('NOT', 14),
 ('accused', 14),
 ('Department', 14),
 ('my', 14),
 ('Address', 14),
 ('Street', 14),
 ('BY', 13),
 ('The', 13),
 ('i', 13),
 ('State', 13),
 ('$', 13),
 ('11', 13),
 ('.1', 13),
 ('Birth', 13),
 ('A', 12),
 ('Investigation', 12),
 ('Section', 12),
 ('be', 12),
 ('as', 12),
 ('with', 12),
 ('investigation', 12),
 ('Inventory', 12),
 ('Complaint', 11),
 ('no', 11),
 ('4', 11),
 ('this', 11),
 ('on', 11),
 ('Residence', 11),
 ('Chicago', 11),
 ('DISTRICT', 11),
 ('Found', 11),
 ('Disposition', 11),
 ('RECOMMEND', 10),
 ('OPTIONS', 10),
 ('BE', 10)

As we can see, we get a lot of useless results, filled with stop words. Let's try using count vectorizer to filter out stop words and maybe ignore words that are too frequent.
    - cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
    - max_df — When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). This is to ensure that we only have words relevant to the context and not commonly used words.
    - max_features — determines the number of columns in the matrix.
    - n-gram range — we would want to look at a list of single words, two words (bi-grams) and three words (tri-gram) combinations.
    
Resource:
[Automated Keyword Extraction from Articles using NLP](https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34)

In [32]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(split_by_space_punc(text1))
list(cv.vocabulary_.keys())[:10]

['command',
 'channel',
 'review',
 'sasso',
 'emp',
 '14982',
 'nw',
 '32007',
 'date',
 'initiated']

Well... it seems like there are just too much noise in the data. Let's see if we can find a part of the text that just describes the allegation.

In [94]:
matcher2 = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched
terminology_list2 = ["Narrative of", "signed"]
patterns2 = [nlp.make_doc(text) for text in terminology_list2]
matcher2.add("narrative", None, *patterns2)

matches2 = matcher2(doc)#print the matched results and extract out the results
for match_id, start, end in matches2:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]  # The matched span
    print("Indexes: {} to {}, Word: {}".format(start, end, span.text))
    #print("Category: {}     Word: {}".format(string_id, span.text))

Indexes: 1206 to 1207, Word: Signed
Indexes: 1842 to 1844, Word: NARRATIVE OF
Indexes: 1931 to 1932, Word: SIGNED


In [34]:
text_narrative = doc[1844:1931]
text_narrative

 The complainant alleges he was searched three times without justi?cation at the ?rst location as he took out the garbage. The complainant alleges at the second location he was searched again, drugs were mysteriously found and he was falsely arrested. The complainant further alleges his State Identi?cation Card, house keys, wallet containing miscellaneous papers and $60.00 U.S.C. were taken and not inventoried or returned. The complainant states the case was dismissed on 10 AUG 06. WVES .

Looks like we need to use some lemmatiziation. Let's lemmatize the docs and phrases that we are using to match.

In [36]:
doc2 = ' '.join([token.lemma_ for token in text_narrative])
doc2

'  the complainant allege -PRON- be search three time without justi?cation at the ? rst location as -PRON- take out the garbage . the complainant allege at the second location -PRON- be search again , drug be mysteriously find and -PRON- be falsely arrest . the complainant further allege -PRON- State identi?cation card , house key , wallet contain miscellaneous paper and $ 60.00 U.S.C. be take and not inventorie or return . the complainant state the case be dismiss on 10 AUG 06 . WVES .'

In [37]:
doc2 = ' '.join([token.lemma_ for token in doc])
doc2

'COMMAND channel review a Sasso Emp 14982 NW 1 32007 date initiate COMPLAINT REGISTER investigation no.2 CHICAGO POLICE DEPARTMENT 315343 12 MARCH 200 ? investigator SECTION T0 : unit : command Of?cer Unit IZI - GIS please submit this command channel review FORM through channel with the case file to : INTERNAL AFFAIRS DIVISION OFFICE of PROFESSIONAL STANDARDS date receive by unit -PRON- concur [ a recommend option be grant not recommend option not be grant reviewed by TITLE a   unit - date forward by unit 0 ? Wrdy a 7 date receive by unit   -PRON- concur -PRON- do not   14/4457 : 70 % ? recommend option be grant CI recommend option not be GRANTED REVIEWED BYZ unit 27/ Maj/6 ; ? 4/ Dem/{Z/ 54 ?   date forward by unit date RECEIWY unit [ 3 -PRON- concur recommend option not be grant -PRON- do not recommend option SE GRANTED reviewed by I title unit -PRON- -PRON- date forward by unit date receive by unit -PRON- concur recommend option not be grant -PRON- do not recommend option be grant r

In [38]:
def split_by_punc_lemma(text):
    if isinstance(text, str):
        doc = nlp(text)
    elif is_instance(text, spacy.tokens.doc.Doc):
        doc = text
    else:
        return
    doc2 = ' '.join([token.lemma_ for token in doc])
    return [x.strip() for x in re.split('[,/]', doc2)]

In [39]:
split_by_punc_lemma("searching minors/patting down minors/arresting minors")

['search minor', 'pat down minor', 'arrest minor']

In [40]:
matcher3 = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched

In [41]:
for t in terminology_list:
    print(t, ": ", split_by_punc_lemma(t))

Nudity, Penetration :  ['nudity', 'penetration']
sexual harassment, Sexual remarks :  ['sexual harassment', 'sexual remark']
Sexual humiliation, Sexual extortion, Prostitution/sex work :  ['sexual humiliation', 'sexual extortion', 'prostitution', 'sex work']
Taser, Baton, physical touch, gun :  ['Taser', 'Baton', 'physical touch', 'gun']
Trespass/robbery, Racial slurs, xenophobic remarks :  ['trespass', 'robbery', 'racial slur', 'xenophobic remark']
Undocumented status, ICE :  ['undocumented status', 'ice']
Planting drug, planting guns :  ['plant drug', 'plant gun']
Neglect of duty, failure to serve :  ['neglect of duty', 'failure to serve']
Refusing to provide medical assistance :  ['refuse to provide medical assistance']
Irrational, Aggressive, Unstable :  ['irrational', 'aggressive', 'unstable']
searching minors/patting down minors/arresting minors :  ['search minor', 'pat down minor', 'arrest minor']


In [43]:
for t in terminology_list:
    terms = split_by_punc_lemma(t)
    patterns = [nlp.make_doc(text) for text in terms]
    matcher3.add(t, None, *patterns)

In [44]:
matches3 = matcher3(doc)#print the matched results and extract out the results
for match_id, start, end in matches3:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]  # The matched span
    #print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))
    print("Category: {}     Word: {}".format(string_id, span.text))

Category: Irrational, Aggressive, Unstable     Word: irrational
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: ROBBERY
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: Trespass


In [45]:
test = nlp("falsely")
doc2 = ' '.join([token.lemma_ for token in test])
doc2

'falsely'

There are two types of stemmers in NLTK: Porter Stemmer and Snowball stemmers. 

In [46]:
import nltk

from nltk.stem.porter import *
stemmer = PorterStemmer()

In [47]:
stemmer.stem("falsely")

'fals'

In [64]:
def split_by_space_stop_punc_stem(text):
    words = [x.strip() for x in re.split('\W+', text)]
    new_words = []
    for w in words:
        if not nlp.vocab[w].is_stop:
            new_words.append(stemmer.stem(w))
    return new_words

def get_stemmed_text(text):
    return " ".join(split_by_space_stop_punc_stem(text))

In [65]:
text1_stemmed = get_stemmed_text(text1)
text1_stemmed

'command channel review sasso emp 14982 NW 1 32007 date initi complaint regist investig 2 chicago polic depart 315343 12 march 200 investig section T0 unit command cer unit izi gi submit command channel review form channel case file intern affair divis offic profession standard date receiv unit concur recommend option grant recommend option grant review titl unit date forward unit 0 wrdi 7 date receiv unit concur 14 4457 70 recommend option grant CI recommend option grant review byz unit 27 maj 6 4 dem Z 54 date forward unit date receiwi unit 3 concur recommend option grant recommend option SE grant review titl unit date forward unit date receiv unit concur recommend option grant recommend option grant review titl unit date forward unit date receiv unit concur 3 recommend option grant CI El recommend option SE grant review titl unit date forward unit separ report propos altern find altern recommend requir check C R 315343 intern affair divis 2 march 2007 gener investig section command 

In [50]:
# split by puncuation and stem
def split_by_punc_stem(text):
    phrases = [x.strip() for x in re.split('[,/]', text)]
    return [' '.join([stemmer.stem(s) for s in phrase.split()]) for phrase in phrases]

In [51]:
split_by_punc_stem("searching minors/patting down minors/arresting minors")

['search minor', 'pat down minor', 'arrest minor']

In [52]:
matcher4 = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched
for t in terminology_list:
    print(t, ": ", split_by_punc_stem(t))

Nudity, Penetration :  ['nuditi', 'penetr']
sexual harassment, Sexual remarks :  ['sexual harass', 'sexual remark']
Sexual humiliation, Sexual extortion, Prostitution/sex work :  ['sexual humili', 'sexual extort', 'prostitut', 'sex work']
Taser, Baton, physical touch, gun :  ['taser', 'baton', 'physic touch', 'gun']
Trespass/robbery, Racial slurs, xenophobic remarks :  ['trespass', 'robberi', 'racial slur', 'xenophob remark']
Undocumented status, ICE :  ['undocu statu', 'ice']
Planting drug, planting guns :  ['plant drug', 'plant gun']
Neglect of duty, failure to serve :  ['neglect of duti', 'failur to serv']
Refusing to provide medical assistance :  ['refus to provid medic assist']
Irrational, Aggressive, Unstable :  ['irrat', 'aggress', 'unstabl']
searching minors/patting down minors/arresting minors :  ['search minor', 'pat down minor', 'arrest minor']


In [53]:
for t in terminology_list:
    terms = split_by_punc_stem(t)
    patterns = [nlp.make_doc(text) for text in terms]
    matcher4.add(t, None, *patterns)

In [54]:
doc_stemmed = nlp(text1_stemmed)
matches4 = matcher4(doc_stemmed)#print the matched results and extract out the results
for match_id, start, end in matches4:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc_stemmed[start:end]  # The matched span
    #print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))
    print("Category: {}     Word: {}".format(string_id, span.text))

Category: Planting drug, planting guns     Word: plant drug
Category: Planting drug, planting guns     Word: plant drug
Category: Irrational, Aggressive, Unstable     Word: irrat
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: robberi
Category: Trespass/robbery, Racial slurs, xenophobic remarks     Word: trespass


In [55]:
allegation_name_set = set(document_tags['allegation_name'])

print(allegation_name_set)
print(len(allegation_name_set))

{'Escape', 'Improper Detention', 'Failure To Ensure - Civil Rights', 'Miscellaneous', 'Street', 'Solicit / Accept Bribe (Non-Traffic)', 'Search - Person / Property', 'Unnecessary Display Of Weapon / On Duty', 'Kicked', 'Taser (Probe Discharge)', 'Secondary/Special Employment', 'Just Cause', 'State Civil Suit', 'Firearm Discharge No Hits - Handgun', 'Knee Strike', 'Auto Theft', 'Theft', 'Domestic Altercation - Physical Abuse', 'Altercation / Disturbance - Traffic', 'Sexual Orientation', 'Misconduct During Issuance Of Citation', 'Take Down (Thrown To Ground)', 'Inadequate / Failure To Provide Service', 'Motor Vehicle Fatality - On Duty', 'Use Of Official Position', 'Insubordination', 'Improper Processing / Reporting / Procedures', 'U Converted To C.R. (Records Keeping Only, Initial)', 'Use / Abuse Drugs / Controlled Substance - Off Duty', 'Shoplifting', 'Excessive Force / Off Duty - Injury', 'Threats', 'Assault / Battery, Etc.', 'Harassment', 'Misuse Of Department Records', 'Telephone / 

In [56]:
allegation_category_set = set(document_tags['allegation_category'])

print(allegation_category_set)
print(len(allegation_category_set))
allegation_category_set.remove(None)
print(len(allegation_category_set))

{'Operation/Personnel Violations', 'Criminal Misconduct', 'Verbal Abuse', 'Racial Profiling', 'Traffic', None, 'Illegal Search', 'Unknown', 'Conduct Unbecoming (Off-Duty)', 'Bribery / Official Corruption', 'Use Of Force', 'Supervisory Responsibilities', 'Lockup Procedures', 'False Arrest', 'Excessive Force', 'Domestic', 'Medical', 'Drug / Alcohol Abuse'}
18
17


In [57]:
terminology_list2 = list(allegation_category_set)
matcher5 = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched
for t in terminology_list2:
    print(t, ": ", split_by_punc_stem(t))

Operation/Personnel Violations :  ['oper', 'personnel violat']
Criminal Misconduct :  ['crimin misconduct']
Verbal Abuse :  ['verbal abus']
Racial Profiling :  ['racial profil']
Traffic :  ['traffic']
Illegal Search :  ['illeg search']
Unknown :  ['unknown']
Conduct Unbecoming (Off-Duty) :  ['conduct unbecom (off-duty)']
Bribery / Official Corruption :  ['briberi', 'offici corrupt']
Use Of Force :  ['use Of forc']
Supervisory Responsibilities :  ['supervisori respons']
Lockup Procedures :  ['lockup procedur']
False Arrest :  ['fals arrest']
Excessive Force :  ['excess forc']
Domestic :  ['domest']
Medical :  ['medic']
Drug / Alcohol Abuse :  ['drug', 'alcohol abus']


In [58]:
for t in terminology_list2:
    terms = split_by_punc_stem(t)
    patterns = [nlp.make_doc(text) for text in terms]
    matcher5.add(t, None, *patterns)

In [59]:
doc_stemmed = nlp(text1_stemmed)
matches5 = matcher5(doc_stemmed)#print the matched results and extract out the results
for match_id, start, end in matches5:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc_stemmed[start:end]  # The matched span
    #print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))
    print("Category: {}     Word: {}".format(string_id, span.text))

Category: Drug / Alcohol Abuse     Word: drug
Category: False Arrest     Word: fals arrest
Category: Medical     Word: medic
Category: Medical     Word: medic
Category: Unknown     Word: unknown
Category: Drug / Alcohol Abuse     Word: drug
Category: False Arrest     Word: fals arrest
Category: Domestic     Word: domest
Category: Drug / Alcohol Abuse     Word: drug
Category: False Arrest     Word: fals arrest
Category: Domestic     Word: domest
Category: Medical     Word: medic
Category: Medical     Word: medic
Category: Medical     Word: medic
Category: Drug / Alcohol Abuse     Word: drug


In [63]:
text_narrative_str = str(text_narrative)
text_narrative_str

' The complainant alleges he was searched three times without justi?cation at the ?rst location as he took out the garbage. The complainant alleges at the second location he was searched again, drugs were mysteriously found and he was falsely arrested. The complainant further alleges his State Identi?cation Card, house keys, wallet containing miscellaneous papers and $60.00 U.S.C. were taken and not inventoried or returned. The complainant states the case was dismissed on 10 AUG 06. WVES .'

Now let's try stemming this text and then see what we get by running the matcher.

In [67]:
text_narrative_stemmed = get_stemmed_text(text_narrative_str)
text_narrative_stemmed

' complain alleg search time justi cation rst locat took garbag complain alleg second locat search drug mysteri found fals arrest complain alleg state identi cation card hous key wallet contain miscellan paper 60 00 U S C taken inventori return complain state case dismiss 10 aug 06 wve '

In [68]:
doc_narrative_stemmed = nlp(text_narrative_stemmed)
matches6 = matcher5(doc_narrative_stemmed)#print the matched results and extract out the results
for match_id, start, end in matches6:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc_narrative_stemmed[start:end]  # The matched span
    #print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))
    print("Category: {}     Word: {}".format(string_id, span.text))

Category: Drug / Alcohol Abuse     Word: drug
Category: False Arrest     Word: fals arrest
